<a href="https://colab.research.google.com/github/Guliko24/CE807_Text_Analytics/blob/main/Lab05/lab05_ipynb_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 05: Text Classification using Naive Bayes Classifier

The code assumes that you have data in './CE807-24-SU/Lab05/data' folder, copy the data from the moodle page

In [26]:
import os
import numpy
import io # modified for Python 2.7 MP
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
# from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score

In [27]:
from google.colab import drive
drive.mount('/content/gdrive') # To mount the GDrive, this will ask permission for access

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = os.path.join('./CE807-24-SU/Lab05/')
GOOGLE_DRIVE_PATH = os.path.join('gdrive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print('List files: ', os.listdir(GOOGLE_DRIVE_PATH))


List files:  ['data', 'lab05.ipynb.json']


# Let's read the data

In [29]:
sample_file = os.path.join(GOOGLE_DRIVE_PATH,'data','sample.csv')

data = pd.read_csv(sample_file)


In [30]:
data.head()

,text,class
0,"Date: Wed, 02 Oct 2002 11:39:13 -05...",ham
1,"So, last night around 5:30AM I'm woken up by a...",ham
2,"On Wed, 2002-09-18 at 17:50, Matthias Saou wro...",ham
3,"URL: http://www.newsisfree.com/click/-5,829698...",ham
4,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",spam


In [31]:
data['class'].unique()
# The dataset contain a given text in harmful or spam.

array(['ham', 'spam'], dtype=object)

In [32]:
data['class'].value_counts()

class
ham     2750
spam     500
Name: count, dtype: int64

Building Vocabulary

In [33]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer() # Note no pre-processing is done here. In practice you will have different preprocessing steps.
counts = count_vectorizer.fit_transform(data['text'].values)

Declare and Train NB Model

In [34]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB()

Now Test on some sample data

In [35]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts=count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

Use `pipeline` to make things systematic

In [36]:
pipeline = Pipeline([
    # ('lema',   lemma()), # explore how to add lemmatization or other preprocessing steps in the pipeline
    ('count_vectorizer',   CountVectorizer()),
    ('classifier',         MultinomialNB())
])

Train using `pipeline`

In [37]:
pipeline.fit(data['text'].values, targets)

Pipeline(steps=[('count_vectorizer', CountVectorizer()),
                ('classifier', MultinomialNB())])

Test using `pipeline`

In [38]:
predictions = pipeline.predict(examples)
predictions

array(['spam', 'ham'], dtype='<U4')

**Your Turn**

* Explore Data
* Add your function in the pipeline like stemming
* Different NB classifier like Gaussian, Bernoulli etc
  * You might have to change Vectorization method
* Use 5-fold validation by looking at the documentation
* Figure out how to save and load model


In [39]:
from sklearn.metrics import confusion_matrix, f1_score
# In Lecture 6, you will learn how to select a performance metrics for evaluation
from sklearn.model_selection import train_test_split



In [40]:
import nltk
nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Let's evaluate the model. First we will split the data into `train` and `test` set and use it for evaluation

In [41]:
train_text, test_text, train_class, test_class = train_test_split(data['text'],
                                                                  data['class'],
                                                                  test_size=0.2,
                                                                  random_state=42)

print('No. of Training Data', len(train_text))
print('No. of Testing Data', len(test_text))
# Training on the train data
pipeline.fit(train_text, train_class)

print('Training Done')

# Testing on the test data
predictions= pipeline.predict(test_text)
print('Testing Done')



No. of Training Data 2600
No. of Testing Data 650
Training Done
Testing Done


Model Evaluation

In [42]:
# Now evaluate the model's performance

confusion=confusion_matrix(test_class, predictions)

score=f1_score(test_class, predictions, average='macro')

print('Confusion Matrix')
print(confusion)

print('Micro-F1 Score')
print(score)


Confusion Matrix
[[546   0]
 [ 16  88]]
Micro-F1 Score
0.9511131167268351


**Your Turn**


*   see documentation of `sklearn.metrics` and print different metrics like accuracy, recall, precision and different F1-Score



In [43]:
from sklearn.metrics import precision_score

def calculate_macro_precision(y_true, y_pred, class_label):
    # Calculate precision for the specified class
    precision = round(precision_score(y_true, y_pred, labels=[class_label], average='macro'),2)
    return precision



Your Turn

    Explore Data
    Add your function in the pipeline like stemming
    Different NB classifier like Gaussian, Bernoulli etc
        You might have to change Vectorization method
    Use 5-fold validation by looking at the documentation
    Figure out how to save and load model



In [47]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')


# Initialize stop words and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Function for preprocessing text
def preprocess_and_stem_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Removing numbers
    text = re.sub(r'\d+', '', text)
    #removing "handles"
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    # Tokenization
    words = word_tokenize(text)
    # Removing stop words and stemming
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    # Rejoin words into a single string
    return ' '.join(words)

# Apply preprocessing to the text data
data['cleaned_text'] = data['text'].apply(preprocess_and_stem_text)



# Initialize the CountVectorizer
count_vectorizer = CountVectorizer()

# Fit and transform the cleaned data
counts = count_vectorizer.fit_transform(data['cleaned_text'].values)

# Convert the result to an array and display it
counts_array = counts.toarray()
print(counts_array)

# Display the feature names
feature_names = count_vectorizer.get_feature_names_out()
print(feature_names)


[[0 1 1 1 1 1 0 1 0 0]
 [0 0 1 0 1 1 1 0 1 0]
 [1 0 1 1 1 1 0 0 0 1]]
['bright' 'brown' 'dog' 'fox' 'jump' 'lazi' 'never' 'quick' 'quickli'
 'summer']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Your Turn**


*   Use the `train.csv` to train your model
*   Use the `valid.csv` to validation the model's performance and find best parameters, if any
*   Use the `test.csv` to finally test the model's performance






In [45]:
train_file = os.path.join(GOOGLE_DRIVE_PATH,'data','train.csv')

data_train = pd.read_csv(train_file)
data_train.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
